# 定点和浮点

$n$ 位 $p$ 进制自然数可以表示为：

$$
\begin{aligned}
x &= x_0 x_1 \cdots x_{n-1} x_{n} . x_{-1} \cdots x_{-n+1} x_{-n}\\
&= (x_0, x_1, \cdots, x_{n-1}, x_{n}, x_{-1}, \cdots, x_{-n+1}, x_{-n}) \begin{bmatrix}
p^n \\ p^{n-1} \\ \vdots \\ p^1 \\ p^0 \\ p^{-1} \\ \vdots \\ p^{-n+1} \\ p^{-n}
\end{bmatrix}
\end{aligned}
$$ (FP1)

这里 $p=2$。

定点和浮点都是数值的表示，它们区别在于，将整数（integer）部分和小数（fractional）分开的点在哪里。定点保留特定位数整数和小数，而浮点保留特定位数的有效数字（significand）和指数（exponent）。

## 定点数表示法

定点数指的是小数点固定的存储方式，因为小数点固定，所以不需要额外使用存储空间存储小数点，只需要约定好即可。计算机中定点数有两种类型：

`定点小数`
:   纯小数，约定的小数点位置在符号位之后、有效数值部分最高位之前。

    若数据 $x=s_0.x_{-1} x_{-2} \cdots x_{-n}$，其中 $s_0$ 为符号位，$x_{-1} \cdots x_{-n}$ 为数值有效部分，也称为 **尾数**；$x_{-1}$ 为最高有效位。

    一般说来，如果最末位 $x_{-n} = 1$，前面各位都为 $0$，则数的绝对值最小，即 $|x|_{\min} = 2^{-n}$。如果各位均为 $1$，则数的绝对值最大，即 $|x|_{\max} = 1-2^{-n}$。所以定点小数的表示范围是：$2^{-n} \leq |x| \leq 1 - 2^{-n}$。

`定点整数`
:   纯整数，约定的小数点位置在有效数值部分最低位之后。

    若数据 $x=s_0x_1 x_2 \cdots x_n$，其中 $s_0$ 为符号位，$x_1 \cdots x_n$ 为数值有效部分，也称为 **尾数**；$x_1$ 为最高有效位。

    定点整数的表示范围是：$1 \leq |x| \leq 2^{n} - 1$。

当数据小于定点数能表示的最小值时，计算机将它们作 $0$ 处理，称为 **下溢**；大于定点数能表示的最大值时，计算机将无法表示，称为 **上溢**，上溢和下溢统称为 **溢出**。

计算机采用定点数表示时，对于既有整数又有小数的原始数据，需要设定一个缩放因子，数据按其缩小成定点小数或扩大成定点整数再参加运算，运算结果，根据缩放因子，还原成实际数值。若缩放因子选择不当，往往会使运算结果产生溢出或降低数据的有效精度。

用定点数进行运算处理的计算机被称为 **定点机**。

## 定点数实例

可以使用 {func}`bin` 将整数转换为二进制表示。

In [1]:
bin(0), bin(1), bin(2), bin(3), bin(4), bin(5), bin(6)

('0b0', '0b1', '0b10', '0b11', '0b100', '0b101', '0b110')

对于 `25.125` 表示为定点整数需要如下操作：

1. 将整数部分转换为定点整数：

In [2]:
bin(25)

'0b11001'

In [3]:
def fixed_inter(x):
    L = []
    while x != 0:
        L.append(str(x % 2))
        x = x // 2
    L.reverse()
    x = "0b" + "".join(L)
    return x

In [4]:
fixed_inter(25)

'0b11001'

2. 小数部分转换为定点小数：

In [5]:
import numpy as np


def fixed_decimal(x):
    L = []
    while x != 0:
        x, b = np.modf(x * 2)
        L.append(str(int(b)))
    x = "." + "".join(L)
    return x

In [7]:
x = 0.125
fixed_decimal(x)

'.001'

拼在一起就是：

In [8]:
fixed_inter(25) + fixed_decimal(x)

'0b11001.001'





## 浮点数表示法

与科学计数法相似，任意 $J$ 进制数 $N$，总可以写成：

$$
N = J^E \times M
$$

式中 $M$ 称为数 $N$ 的 **尾数** （mantissa），是一个纯小数；$E$ 为数 $N$ 的 **阶码** （exponent），是一个整数，$J$ 称为缩放因子 $J^E$ 的底数。这种表示方法相当于数的小数点位置随缩放因子的不同而在一定范围内可以自由浮动，所以称为 **浮点表示法**。

底数是事先约定好的（常取 2），在计算机中不出现。在机器中表示一个浮点数时，一是要给出尾数，用定点小数形式表示。尾数部分给出有效数字的位数，因而决定了浮点数的表示精度。二是要给出阶码，用整数形式表示，阶码指明小数点在数据中的位置，因而决定了浮点数的表示范围。浮点数也要有符号位。因此一个机器浮点数应当由阶码和尾数及其符号位组成：

$$
x = E_s E_1 E_2 \cdots E_n M_s M_1 M_2 \cdots M_n
$$

其中 $E_s$ 是阶符（表示阶码的符号），$E_1 E_2 \cdots E_n$ 的阶码，$M_s$ 是尾符（数 $N$ 的符号），$M_1 M_2 \cdots M_n$ 的尾数。

当底数取 2 时，二进制数 $N$ 的小数点每右移一位，阶码减小 $1$，相应尾数右移一位；反之，小数点每左移一位，阶码加 $1$，相应尾数左移一位。

为了提高数据的表示精度，当尾数的值不为 $0$ 时，其绝对值应大于等于 $0.5$，即尾数域的最高有效位应为 $1$，否则要以修改阶码同时左右移小数点的方法，使其变成这一要求的表示形式，这称为 **浮点数的规格化表示**。

当浮点数的尾数为 $0$ 时，不论其阶码为何值，或者当阶码的值遇到比它能表示的最小值还小时，不管其尾数为何值，计算机都把该浮点数看成 $0$ 值，称为 **机器零**。

浮点数所表示的范围比定点数大。假设机器中的数由 $8$ 位二进制数表示（包括符号位）：在定点机中这 $8$ 位全部用来表示有效数字（包括符号）；在浮点机中若阶符、阶码占 $3$ 位，尾符、尾数占 $5$ 位，在此情况下，若只考虑正数值，定点机小数表示的数的范围是 $0.0000000$ 到 $0.1111111$，相当于十进制数的 $0$ 到 $127 ／128$，而浮点机所能表示的数的范围则是 $2^{-11} \times 0.0001$ 到 $2^{11} \times 0.1111$，相当于十进制数的 $1／128$ 到 $7.5$。显然，都用 $8$ 位，浮点机能表示的数的范围比定点机大得多。

```{note}
IEEE 754 浮点数计数标准，可以表示为：

$$
x = 1.M... \times 2^E
$$
```

因此，只要给出：符号（$S$）、阶码部分（$E$）、尾数部分（$M$） 这三个维度的信息，一个浮点数的表示就完全确定下来了，所以 float 和 double 这两种类型的浮点数在计算机中的存储结构就表示成下图所示这个样子：

![](images/float_point.png)

- 对于 float 型浮点数，指数部分 $8$ 位，考虑可正可负，因此可以表示的指数范围为 $[-127, 128]$；
- 对于 double 型浮点数，指数部分 $11$ 位，考虑可正可负，因此可以表示的指数范围为 $[-1023, 1024]$。

浮点数的精度是由尾数的位数来决定的：

- 对于 float 型浮点数，尾数部分 $23$ 位，换算成十进制就是 $2^{23}=8388608$，所以十进制精度只有 $6 ~ 7$ 位；
- 对于 double 型浮点数，尾数部分 $52$ 位，换算成十进制就是 $2^{52} = 4503599627370496$，所以十进制精度只有 $15 ~ 16$ 位。

所以，浮点数交给计算机存储的时候，可能会有精度丢失问题！

在指令集（Instruction Set Architecture）的内置数据类型中，定点数是整数，浮点是二进制格式。一般来说，指令集层面的定点是连续的，因为它是整数，且两个邻近的可表示数字的间隙是 $1$。另一方面，浮点代表实数，其数值间隙由指数确定，因而具有非常宽的值域（32 位数值最大整数是 $2^{31}-1$，而浮点值域为 $2 - 2^{-23} \times 2^{127}$，值越接近零就越准确。在给定指数时，浮点在不同范围内拥有数值数量相同数量。

